In [ ]:
# Import Space
import cv2
import sys
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import time
import math
import imageio
import glob, os
import datetime
import heapq
import numpy as np
from scipy.signal import find_peaks_cwt
from matplotlib.patches import Ellipse


def smooth(x, window_len=11, window='hanning'):
    """
    Acceptable inputs for window variable:
        'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")

    if window_len < 3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError(
            "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
        )

    s = np.r_[x[window_len - 1:0:-1], x, x[-2:-window_len - 1:-1]]
    
    if window == 'flat':  #moving average
        w = np.ones(window_len, 'd')
    else:
        w = eval('np.' + window + '(window_len)')

    y = np.convolve(w / w.sum(), s, mode='valid')
    return y


# Return vector with zeroes in place of of values outside min/max bounds
def notchFilter(inputVec, minVec, maxVec=sys.maxsize):
    returnVec = np.copy(inputVec)
    for i in range(inputVec.size):
        if inputVec[i] < minVec or inputVec[i] > maxVec:
            returnVec[i] = 0
    return returnVec


# Count number of nonzero intervals in signal
def peakNumber(cleanVector, minZeroes=1):
    numPeaks = 0
    count = 0
    peak = True
    for datapoint in cleanVector:
        if (datapoint > 0 and peak):
            print(datapoint)
            numPeaks += 1
            peak = False
            count = 0
        else:
            if (datapoint == 0):
                count += 1
            if (count >= minZeroes):
                peak = True
    return numPeaks


def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scipy.signal.butter(
        order, normal_cutoff, btype='high', analog=False)
    return b, a

# Attenuate low frequencies, pass high.
def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = scipy.signal.filtfilt(b, a, data)
    return y


def butter_lowpass(cutOff, fs, order=5):
    nyq = 0.5 * fs
    normalCutoff = cutOff / nyq
    b, a = scipy.signal.butter(order, normalCutoff, btype='low', analog=True)
    return b, a

# Attenuate high frequencies, pass low.
def butter_lowpass_filter(data, cutOff, fs, order=4):
    b, a = butter_lowpass(cutOff, fs, order=order)
    y = scipy.signal.lfilter(b, a, data)
    return y

# Find aTan2 of y,x vector. Returns degrees within 0-360.
def angle(y, x):
    return ((57.2958 * math.atan2(y, x) + 360) % 360)


In [ ]:
gui = np.zeros((300,512,3), np.uint8)
cv2.namedWindow('gui')

def nothing(x):
    pass

filename = 'recordings/9.avi'
cap = cv2.VideoCapture(filename)
for i in range(1):
    ret, frame = cap.read()
height = frame.shape[0] 
width = frame.shape[1]

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cv2.createTrackbar('h-low','gui',0,255,nothing)
cv2.createTrackbar('h-high','gui',255,255,nothing)
cv2.createTrackbar('s-low','gui',0,255,nothing)
cv2.createTrackbar('s-high','gui',255,255,nothing)
cv2.createTrackbar('v-low','gui',0,255,nothing)
cv2.createTrackbar('v-high','gui',255,255,nothing)
switch = '0 : OFF \n1 : ON'
cv2.createTrackbar(switch, 'gui',0,1,nothing)
s = 0

while(s<0.9):
    cv2.imshow('gui',gui)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
        
    hLow = cv2.getTrackbarPos('h-low','gui')
    hHigh = cv2.getTrackbarPos('h-high','gui')
    sLow = cv2.getTrackbarPos('s-low','gui')
    sHigh = cv2.getTrackbarPos('s-high','gui')
    vLow = cv2.getTrackbarPos('v-low','gui')
    vHigh = cv2.getTrackbarPos('v-high','gui')
    lowerHSV = (hLow, sLow, vLow)
    upperHSV = (hHigh, sHigh, vHigh)    
    s = cv2.getTrackbarPos(switch,'gui')

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(image, lowerHSV, upperHSV)
    image = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Bitwise-AND mask and original image
    image = cv2.bitwise_and(image,image, mask= mask)
    im2,contours,hierarchy = cv2.findContours(image,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) >= 1:
        contList = sorted(contours, key=cv2.contourArea)
        contList = contList[::-1]
        
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        image = cv2.drawContours(image, contList[0:2], -1, (0,255,0), 3)
    
    cv2.imshow('frame', image)

pos = []
display_frame = frame.copy()


brightarray = []
octo1Size = []
octo2Size = []
brightList = []
cap = cv2.VideoCapture(filename)
ret = True
for i in range(1):
    ret, frame = cap.read()
while(ret):
    ret, frame = cap.read()

    # Break check, remember your seatbelt
    if not ret:
        break
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(image, lowerHSV, upperHSV)
    image = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    image = cv2.bitwise_and(image,image, mask= mask)
    im2,contours,hierarchy = cv2.findContours(image, 1, 2)
    if len(contours) >= 1:
        contList = sorted(contours, key=cv2.contourArea)
        contList = contList[::-1]
        
        #Add sizes of largest contours (octopuses)
        octo1Size.append(cv2.contourArea(contList[0]))
        octo2Size.append(cv2.contourArea(contList[1]))
        
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        image = cv2.drawContours(image, contList[0:2], -1, (0,255,0), 3)
        if(len(contList) < 2 or cv2.contourArea(contList[1])<200):
            brightList.append(1)
            image = cv2.circle(image,(0,0),50,(0,255,0),-1)
        else:
            brightList.append(0)
    
    cv2.imshow('frame', image)    
    
cv2.destroyAllWindows()
cap.release()
plt.plot(brightList)
plt.ylim(0, 2)
plt.show()

In [ ]:
fig, ax = plt.subplots()
octo1Size = np.array(octo1Size)
octo2Size = np.array(octo2Size)

index = np.linspace(0, smooth(octo1Size, 10, window='hanning').size/30,smooth(octo1Size, 10, window='hanning').size)
ax.plot(index, smooth(octo1Size, 10, window='hanning'), label=("Area 1"))
ax.plot(index, smooth(octo2Size, 10, window='hanning'), label=("Area 2"))

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()
index = np.linspace(0, np.gradient(octo1Size).size/30,np.gradient(octo1Size).size)
plt.plot(np.gradient(octo1Size))
plt.show()
print("Contact found at " + str(np.argmax(np.gradient(octo1Size))/30) + " seconds.")

In [ ]:
np.save("goodfilenameforgradient", np.gradient(octo1Size))